# Tweepyでfollowerの共通部分を取得
## API.friends_ids(id/screen_name/user_id[, cursor]) return list of Integers
- **id**がフォローしている人のidを返す
- [Doc](https://kurozumi.github.io/tweepy/api.html?highlight=cursor#API.friends_ids)
- **id** – Specifies the ID or screen name of the user.
- **screen_name** – Specifies the screen name of the user. Helpful for disambiguating when a valid screen name is also a user ID.
- **user_id** – Specifies the ID of the user. Helpful for disambiguating when a valid user ID is also a valid screen name.
- **cursor** – Breaks the results into pages. Provide a value of -1 to begin paging. Provide values as returned to in the response body’s next_cursor and previous_cursor attributes to page back and forth in the list.

## API.followers_ids(id/screen_name/user_id) return list of Integers
- **id**をフォローしている人のidを返す
- [Doc](https://kurozumi.github.io/tweepy/api.html?highlight=cursor#API.followers_ids)
- **id** – Specifies the ID or screen name of the user.
- **screen_name** – Specifies the screen name of the user. Helpful for disambiguating when a valid screen name is also a user ID.
- **user_id** – Specifies the ID of the user. Helpful for disambiguating when a valid user ID is also a valid screen name.
- **cursor** – Breaks the results into pages. Provide a value of -1 to begin paging. Provide values as returned to in the response body’s next_cursor and previous_cursor attributes to page back and forth in the list.


## lookup_users
- 一度に100件までユーザー情報を取得できる
- リファレンスにはないが、ソースコードには存在↓
- [Doc](https://github.com/tweepy/tweepy/blob/fdf133184a918a64a0853dc52bffddc90570dcfa/tweepy/api.py#L310-L321)
```python
def lookup_users(self, user_ids=None, screen_names=None, include_entities=None):
        """ Perform bulk look up of users from user ID or screenname """
        post_data = {}
        if include_entities is not None:
            include_entities = 'true' if include_entities else 'false'
            post_data['include_entities'] = include_entities
        if user_ids:
            post_data['user_id'] = list_to_csv(user_ids)
        if screen_names:
            post_data['screen_name'] = list_to_csv(screen_names)

        return self._lookup_users(post_data=post_data)
```

In [0]:
import tweepy
import pandas as pd
from google.colab import files

In [0]:
def my_auth (consumer_key, consumer_secret, access_token, access_token_secret):
  auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
  auth.set_access_token(access_token, access_token_secret)
  return tweepy.API(auth)

def friends_common_part(api, *args):
  candidate_dict = {}
  if len(args) < 2:
    return "error"
  
  common_set = set(api.friends_ids(args[0]))
  for i in range(1, len(args)):
    common_set = common_set & set(api.friends_ids(args[i]))
  common_list = list(common_set)
  
  for i in range(0, len(common_list), 100):
    for user in api.lookup_users(user_ids=common_list[i:i+100]):
      candidate_dict[user.id] = {}
      candidate_dict[user.id]['name'] = user.name
      candidate_dict[user.id]['twitter_page'] = 'https://twitter.com/' + user.screen_name
      candidate_dict[user.id]['my_page'] = user.url
      candidate_dict[user.id]['description'] = user.description
  return candidate_dict

def followers_common_part(api, *args):
  candidate_dict = {}
  if len(args) < 2:
    return "error"
  
  common_set = set(api.friends_ids(args[0]))
  for i in range(1, len(args)):
    common_set = common_set & set(api.followers_ids(args[i]))
  
  common_list = list(common_set)
  
  for i in range(0, len(common_list), 100):
    for user in api.lookup_users(user_ids=common_list[i:i+100]):
      candidate_dict[user.id] = {}
      candidate_dict[user.id]['name'] = user.name
      candidate_dict[user.id]['twitter_page'] = 'https://twitter.com/' + user.screen_name
      candidate_dict[user.id]['my_page'] = user.url
      candidate_dict[user.id]['description'] = user.description
  return candidate_dict

↓ あまり実行しすぎると 15分の通信制限がかかるので注意
（APIを叩く処理のみ）

In [0]:
# Twitter Developer Account Credentials
consumer_key = 'hogehoge'
consumer_secret = 'hogehoge'
access_token = 'hogehoge'
access_token_secret = 'hogehoge'
  
# 影響力のある人のID（複数人可能）
user_ids = [
    'genya0407',
#     'EbiEbiEvidence', #ebiebi消えたっぽい
    'ko_watanabe_tw',
    'alohat_jp'
]

#上アカウントがフォローしている人の共通部分
candidate_friends = friends_common_part(
    my_auth(consumer_key, consumer_secret, access_token, access_token_secret),
    *user_ids
)

↓表の表示（API関係なし）

In [0]:
candidate_friends_df = pd.DataFrame(candidate_friends).T[['name','twitter_page','description','my_page']]
candidate_friends_df

,name,twitter_page,description,my_page
1055057801951596544,tatatatataro,https://twitter.com/taro_jxv_happy,🎾🏂🍻🥟🇳🇵☀️,https://t.co/4u3d2irH9h
1112714694588928000,社会的ミーくん,https://twitter.com/knock_9999,三茶の殺し屋 500yen/man,None
1113768585577488384,Shota Hashimoto,https://twitter.com/shotaH_tw,見習いengineer/figure skate お気軽にフォローしてくださいませ〜,None
1113818641336688641,NakaS,https://twitter.com/StartedTwittter,何かになりたい渋谷のエンジニア,None
15312900,chokudai(高橋 直大)🍆🍡🌸,https://twitter.com/chokudai,AtCoder(株)代表取締役社長(競技プログラミングの会社)/筑駒中高→慶應SFC/競プロ...,http://t.co/3temA9ZeaS
521513445,ばんり / Banri Yanahama,https://twitter.com/banri_yanahama,京大院卒 / DeNA / エンジニア / 機械学習,None
1112926435977523201,はっしー,https://twitter.com/kure4_Mars,尋常じゃないくらいゲームしてて気づいたらゲームエンジニアになってたです,None
2819366924,スイ,https://twitter.com/uni745e,AtCoder: 水、Pythonで競プロしてます,None
36297164,えじ｜Amane Suzuki,https://twitter.com/SakuEji,DeNAでデータサイエンティストをしています。Kaggle Master。AtCoder水色...,https://t.co/b3U8GaCsQ0
5787442,yhirano55,https://twitter.com/yoshi_hirano,Freelance Programmer / a.k.a. カルパス / 柴犬 / 音楽: ...,https://t.co/mo3uxmP7pQ


In [0]:
#上アカウントをフォローしている人の共通部分
candidate_followers = followers_common_part(
    my_auth(consumer_key, consumer_secret, access_token, access_token_secret),
    *user_ids
)

In [0]:
candidate_followers_df = pd.DataFrame(candidate_followers).T[['name','twitter_page','description','my_page']]
candidate_followers_df

,name,twitter_page,description,my_page
1113768585577488384,Shota Hashimoto,https://twitter.com/shotaH_tw,見習いengineer/figure skate お気軽にフォローしてくださいませ〜,None
1055057801951596544,tatatatataro,https://twitter.com/taro_jxv_happy,🎾🏂🍻🥟🇳🇵☀️,https://t.co/4u3d2irH9h
1112926435977523201,はっしー,https://twitter.com/kure4_Mars,尋常じゃないくらいゲームしてて気づいたらゲームエンジニアになってたです,None
1113818641336688641,NakaS,https://twitter.com/StartedTwittter,何かになりたい渋谷のエンジニア,None
521513445,ばんり / Banri Yanahama,https://twitter.com/banri_yanahama,京大院卒 / DeNA / エンジニア / 機械学習,None
2819366924,スイ,https://twitter.com/uni745e,AtCoder: 水、Pythonで競プロしてます,None
36297164,えじ｜Amane Suzuki,https://twitter.com/SakuEji,DeNAでデータサイエンティストをしています。Kaggle Master。AtCoder水色...,https://t.co/b3U8GaCsQ0
5787442,yhirano55,https://twitter.com/yoshi_hirano,Freelance Programmer / a.k.a. カルパス / 柴犬 / 音楽: ...,https://t.co/mo3uxmP7pQ
2332514653,かちゃ,https://twitter.com/karrrrraage,ガチで渋谷OLやってます / チャイニーズ / アメリカンドッグ・ノンシリコンシャンプー・ク...,None
1883167934,Takumi Karasawa,https://twitter.com/zaki_krsw,"Harada Lab., UTokyo / AI Research Engineer at ...",https://t.co/jIZfJKw9my


↓ CSV書き出し

In [0]:
candidate_friends_df.to_csv("candidate_friends.csv")
files.download("candidate_friends.csv")

In [0]:
candidate_followers_df.to_csv("candidate_followers.csv")
files.download("candidate_followers.csv")